# Import

In [1]:
import csv
import os
import pickle
import re
import sys

import numpy as np
import pandas as pd
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
from gensim.models.ldamodel import LdaModel
from smart_open import open
from stop_words import get_stop_words
from tqdm import tqdm

/opt/conda/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
/opt/conda/lib/python3.10/site-packages/gensim/matutils.py:22: DeprecationWarning: Please use `triu` from the `scipy.linalg` namespace, the `scipy.linalg.special_matrices` namespace is deprecated.
  from scipy.linalg.special_matrices import triu


## Add configuration file

In [2]:
sys.path.append("/home/jovyan/core/config/")

In [3]:
from ALL import config 

## Set condition

In [4]:
tqdm.pandas()
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 50)

In [5]:
data_type = "20News"
vectorize_type = "sentenceBERT"

# Read data

In [6]:
newsgroups_df = pd.read_csv("../../Preprocessing/data/AgNews/master.csv", index_col=0)

In [7]:
with open("../../Preprocessing/data/AgNews/class.csv", mode="r") as f:
    reader = csv.reader(f)
    class_labels = [label for label in reader]

In [8]:
vector_dims = config["vectorize"]["doc2vec"]["dims"]
model_nums = config["clustering"]["gmm"]["max_model_num"]
vector_model_num = config["vectorize"]["doc2vec"]["max_model_num"]
covariance_types = config["clustering"]["gmm"]["covariance_types"]

In [9]:
class Corpus:
    def __init__(self, path=None, texts=None):
        self.path = path
        self.texts = texts
        self.dictionary = Dictionary(texts)
        self.dictionary.filter_extremes()

    def __iter__(self):
        if self.path is not None:
            for line in open(self.path):
                # assume there's one document per line, tokens separated by whitespace
                yield self.dictionary.doc2bow(line.lower().split())
        else:
            for line in self.texts:
                yield self.dictionary.doc2bow(line)
                
    def __len__(self):
        return len(self.texts)

In [10]:
dictionary = pickle.load(open(f"../../Clustering/data/{data_type}/LDA/dictionary.sav", "rb"))
corpus = pickle.load(open(f"../../Clustering/data/{data_type}/LDA/corpus.sav", "rb"))

In [11]:
model_num = 0

In [12]:
lda = LdaModel.load(f"../../Clustering/data/{data_type}/LDA/model/{model_num}")

In [13]:
pyLDAvis.enable_notebook()

vis = gensimvis.prepare(
      lda, corpus, dictionary, n_jobs = 1, sort_topics = False
      )

pyLDAvis.display(vis)

/opt/conda/lib/python3.10/site-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)
